In [125]:
import feedparser
import pandas as pd
import numpy as np
import sqlite3

In [126]:
sources = {'vedomosti': 'https://www.vedomosti.ru/rss/rubric/business', 'lenta.ru': 'https://lenta.ru/rss/news', 'tass': 'https://tass.ru/rss/v2.xml', }

In [203]:
def rss_scrapper_news(sources):  # sources should be a dict with key = name of a resource and val = rss link

    news = []

    for source in sources.keys(): # iterating by keys ('vedomosti', 'lenta.ru' etc.)
        feed = feedparser.parse(sources[source]).entries  #parsing a specific rss feed, e.g. vedomosti

        for entry in feed:  # for each news (entry) in the list
                novost = {'title': entry['title'], 'media': source, 'date': entry['published_parsed'][0:3],
                        'desc': np.nan, 'link': entry['link']}  #create a news by specific structure
                if 'summary' in entry.keys():  # if summary exists, it will replace nan
                    novost['desc'] = entry['summary']
                news.append(novost)
                
    news = pd.DataFrame(news)
    decomp = {'year':[] , 'month':[] , 'day': []}
    for date in news['date']:
        decomp['year'].append(date[0])
        decomp['month'].append(date[1])
        decomp['day'].append(date[2])
    decomp = pd.DataFrame(decomp)
    news = pd.concat([news, decomp], axis=1)
    news['date'] = pd.to_datetime(news[['year', 'month', 'day']])
    news = news.drop(['year', 'month', 'day'], axis=1)

    return news



def save_results(df): 
    try:
        conn = sqlite3.connect('rss.db')
        df.to_csv('rss.csv')
        df.to_sql('RSS', conn)
    except Exception as e:
        print(f'Something went wrong: {e}')
    finally:
        conn.close()
        print('The command has been executed')

In [204]:
df = rss_scrapper_news(sources)

In [205]:
df

,title,media,date,desc,link
0,Минтранс разработал новое положение о регулиро...,vedomosti,2021-09-16,"Операторы опасаются, что правила создадут допо...",https://www.vedomosti.ru/business/articles/202...
1,Поставки серийных Ил-114-300 могут сдвинуться ...,vedomosti,2021-09-16,Это не будет критичным для заказчиков самолето...,https://www.vedomosti.ru/business/articles/202...
2,Польша не хочет продлевать контракт с «Газпром...,vedomosti,2021-09-16,Для монополии не составит проблем договориться...,https://www.vedomosti.ru/business/articles/202...
3,Правительство уведомили о повышении оптовых це...,vedomosti,2021-09-16,Это связано с ростом издержек и высоким спросо...,https://www.vedomosti.ru/business/articles/202...
4,Smartavia планирует запустить рейсы из «Шереме...,vedomosti,2021-09-16,NaN,https://www.vedomosti.ru/business/news/2021/09...
...,...,...,...,...,...
495,Почти 50 тыс. запросов поступают на платформу ...,tass,2021-09-17,По словам председателя Общественного штаба Але...,https://tass.ru/politika/12428893
496,"Почти 2,8 млрд рублей налогов поступило с нача...",tass,2021-09-17,"Это в 3,8 раза превышает поступления за аналог...",https://tass.ru/ekonomika/12428875
497,Врио губернатора Ульяновской области проголосо...,tass,2021-09-17,Он проголосовал в Доме культуры села Карлинско...,https://tass.ru/politika/12428931
498,Международная организация франкофонии приостан...,tass,2021-09-17,Организация потребовала от Национального комит...,https://tass.ru/mezhdunarodnaya-panorama/12428923


In [207]:
save_results(df)

The command has been executed


In [201]:
conn = sqlite3.connect('rss.db')
df1 = pd.read_sql('SELECT * FROM RSS', conn)

In [208]:
df

,title,media,date,desc,link
0,Минтранс разработал новое положение о регулиро...,vedomosti,2021-09-16,"Операторы опасаются, что правила создадут допо...",https://www.vedomosti.ru/business/articles/202...
1,Поставки серийных Ил-114-300 могут сдвинуться ...,vedomosti,2021-09-16,Это не будет критичным для заказчиков самолето...,https://www.vedomosti.ru/business/articles/202...
2,Польша не хочет продлевать контракт с «Газпром...,vedomosti,2021-09-16,Для монополии не составит проблем договориться...,https://www.vedomosti.ru/business/articles/202...
3,Правительство уведомили о повышении оптовых це...,vedomosti,2021-09-16,Это связано с ростом издержек и высоким спросо...,https://www.vedomosti.ru/business/articles/202...
4,Smartavia планирует запустить рейсы из «Шереме...,vedomosti,2021-09-16,NaN,https://www.vedomosti.ru/business/news/2021/09...
...,...,...,...,...,...
495,Почти 50 тыс. запросов поступают на платформу ...,tass,2021-09-17,По словам председателя Общественного штаба Але...,https://tass.ru/politika/12428893
496,"Почти 2,8 млрд рублей налогов поступило с нача...",tass,2021-09-17,"Это в 3,8 раза превышает поступления за аналог...",https://tass.ru/ekonomika/12428875
497,Врио губернатора Ульяновской области проголосо...,tass,2021-09-17,Он проголосовал в Доме культуры села Карлинско...,https://tass.ru/politika/12428931
498,Международная организация франкофонии приостан...,tass,2021-09-17,Организация потребовала от Национального комит...,https://tass.ru/mezhdunarodnaya-panorama/12428923


In [206]:
conn.close()

In [171]:
df